<a href="https://colab.research.google.com/github/herjh0405/DACON_Meal/blob/master/pycaret_water.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pycaret

In [ ]:
from pycaret.regression import *

In [ ]:
!pip install konlpy

In [ ]:
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, log_loss

import pandas as pd
import numpy as np
np.random.seed(0)

from tqdm.notebook import tqdm

import os
import re
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
path = '/content/drive/MyDrive/water/'

train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')


sample_submission = pd.read_csv(path + 'sample_submission.csv')

In [ ]:
from konlpy.tag import Kkma
kkma = Kkma()

In [ ]:
for i in tqdm(range(1205)):       
    train.iloc[:,7][i] = kkma.nouns(train.iloc[:,7][i])
    train.iloc[:,8][i] = kkma.nouns(train.iloc[:,8][i])
    train.iloc[:,9][i] = kkma.nouns(train.iloc[:,9][i])

  0%|          | 0/1205 [00:00<?, ?it/s]

In [ ]:
word_list_morning = []
word_list_lunch = []
word_list_dinner = []
for i in range(1205):
    word_list_morning.append(train.iloc[:,7][i])
    word_list_lunch.append(train.iloc[:,8][i])
    word_list_dinner.append(train.iloc[:,9][i])

In [ ]:
word_morning = []
for i in word_list_morning:
    if i not in word_morning:
        word_morning.append(i)

In [ ]:
word_lunch = []
for i in word_list_lunch:
    if i not in word_lunch:
        word_lunch.append(i)

In [ ]:
word_dinner = []
for i in word_list_dinner:
    if i not in word_dinner:
        word_dinner.append(i)

In [ ]:
food = word_morning + word_lunch + word_dinner

In [ ]:
#dictionary = {food : i for i,food in enumerate(food)}
#print(dictionary)

In [ ]:
#train_df.columns = ['num','datetime','target','temperature','windspeed','humidity','precipitation','insolation','nelec_cool_flag','solar_flag']
#test_df.columns = ['num','datetime','temperature','windspeed','humidity','precipitation','insolation','nelec_cool_flag','solar_flag']

In [ ]:
train['일자'] = pd.to_datetime(train['일자'])
test['일자'] = pd.to_datetime(test['일자'])

In [ ]:
train['month'] = train['일자'].dt.month

test['month'] = test['일자'].dt.month

In [ ]:
train['요일'] = train['요일'].map({'월':0, '화':1, '수':2, '목':3, '금':4})
test['요일'] = test['요일'].map({'월':0, '화':1, '수':2, '목':3, '금':4})

In [ ]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계', 'month'],
      dtype='object')

In [ ]:
def feature_pipeline(df):
    df['남아있는회사원'] = df['본사정원수']-df['본사휴가자수']-df['본사출장자수']-df['현본사소속재택근무자수']
    df['본사휴가자수비율'] = df['본사휴가자수']/df['본사정원수']
    df['본사출장자수비율'] = df['본사출장자수']/df['본사정원수']
    df['추가근무비율'] = df['본사시간외근무명령서승인건수']/df['본사정원수']
    df['본사재택근무비율'] = df['현본사소속재택근무자수']/df['본사정원수']
    
    X_pivot_train = pd.pivot_table(data = df, 
                               values = ['본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수'],  
                               index = '요일', 
                               aggfunc = ['mean', 'min','max', 
                                          'std','var'   
                                         ]
                              )
    
    X_columns = [agg + '_' + '요일' + '_' + column for agg,column in X_pivot_train.columns]
    X_pivot_train.columns = X_columns
    df = pd.merge(df, X_pivot_train, on='요일')

    X_pivot_train = pd.pivot_table(data = df, 
                               values = ['본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수'],  
                               index = 'month', 
                               aggfunc = ['mean', 'min','max', 
                                          'std','var'   
                                         ]
                              )
    
    X_columns = [agg + '_' + 'month' + '_' + column for agg,column in X_pivot_train.columns]
    X_pivot_train.columns = X_columns
    df = pd.merge(df, X_pivot_train, on='month')

    return df

In [ ]:
train= feature_pipeline(train)
test= feature_pipeline(test)

In [ ]:
train_1 = train.drop(['석식계'],axis=1)
train_2 = train.drop(['중식계'],axis=1) 

In [ ]:
reg = setup(data = train_1, 
             target = '중식계',
             numeric_imputation = 'mean',
             normalize = True,
             silent = True)

,Description,Value
0,session_id,3935
1,Target,중식계
2,Original Data,"(1205, 67)"
3,Missing Values,False
4,Numeric Features,44
5,Categorical Features,21
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(843, 730)"


In [ ]:
compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,7.141100e+01,8.990895e+03,9.372010e+01,8.005000e-01,0.1167,0.0872,0.123
gbr,Gradient Boosting Regressor,7.180070e+01,9.094013e+03,9.468550e+01,7.983000e-01,0.1158,0.0873,0.348
rf,Random Forest Regressor,7.231320e+01,9.211372e+03,9.473550e+01,7.959000e-01,0.1168,0.0880,0.882
et,Extra Trees Regressor,7.174650e+01,9.373729e+03,9.536400e+01,7.927000e-01,0.1158,0.0867,0.994
par,Passive Aggressive Regressor,7.373680e+01,9.465728e+03,9.671340e+01,7.885000e-01,0.1203,0.0896,0.172
lasso,Lasso Regression,7.376840e+01,9.516485e+03,9.694380e+01,7.870000e-01,0.1197,0.0895,0.037
br,Bayesian Ridge,7.426760e+01,9.552297e+03,9.718830e+01,7.862000e-01,0.1203,0.0900,0.735
ridge,Ridge Regression,7.469160e+01,9.614766e+03,9.755740e+01,7.848000e-01,0.1208,0.0907,0.038
huber,Huber Regressor,7.446270e+01,9.715527e+03,9.790280e+01,7.828000e-01,0.1214,0.0905,0.342
omp,Orthogonal Matching Pursuit,7.504860e+01,1.034193e+04,1.007118e+02,7.697000e-01,0.1245,0.0909,0.034


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=3935, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
best_5 = compare_models(sort = 'MAE', n_select = 5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,7.141100e+01,8.990895e+03,9.372010e+01,8.005000e-01,0.1167,0.0872,0.077
et,Extra Trees Regressor,7.174650e+01,9.373729e+03,9.536400e+01,7.927000e-01,0.1158,0.0867,1.006
gbr,Gradient Boosting Regressor,7.180070e+01,9.094013e+03,9.468550e+01,7.983000e-01,0.1158,0.0873,0.351
rf,Random Forest Regressor,7.231320e+01,9.211372e+03,9.473550e+01,7.959000e-01,0.1168,0.0880,0.860
par,Passive Aggressive Regressor,7.373680e+01,9.465728e+03,9.671340e+01,7.885000e-01,0.1203,0.0896,0.169
lasso,Lasso Regression,7.376840e+01,9.516485e+03,9.694380e+01,7.870000e-01,0.1197,0.0895,0.039
br,Bayesian Ridge,7.426760e+01,9.552297e+03,9.718830e+01,7.862000e-01,0.1203,0.0900,0.731
huber,Huber Regressor,7.446270e+01,9.715527e+03,9.790280e+01,7.828000e-01,0.1214,0.0905,0.344
ridge,Ridge Regression,7.469160e+01,9.614766e+03,9.755740e+01,7.848000e-01,0.1208,0.0907,0.036
omp,Orthogonal Matching Pursuit,7.504860e+01,1.034193e+04,1.007118e+02,7.697000e-01,0.1245,0.0909,0.036


In [ ]:
blended = blend_models(estimator_list = best_5, fold = 5, optimize = 'MAE')
pred_holdout = predict_model(blended)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,64.3539,7931.7538,89.0604,0.8191,0.1153,0.0807
1,72.7485,8933.5450,94.5174,0.8097,0.1125,0.0859
2,63.4782,6566.5652,81.0343,0.8514,0.1005,0.0781
3,68.1664,8248.0809,90.8189,0.8320,0.1058,0.0801
4,74.9983,10870.2103,104.2603,0.7374,0.1342,0.0926
Mean,68.7490,8510.0310,91.9383,0.8099,0.1137,0.0835
SD,4.5274,1409.2475,7.5752,0.0389,0.0115,0.0052


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,65.8808,8248.3669,90.8205,0.7968,0.1163,0.083


In [ ]:
final_model = finalize_model(blended)

In [ ]:
pred1 = predict_model(final_model,test)

In [ ]:
submission = sample_submission.copy()

In [ ]:
submission['중식계'] = pred1['Label']

In [ ]:
reg = setup(data = train_2, 
             target = '석식계',
             numeric_imputation = 'mean',
             normalize = True,
             silent = True)

,Description,Value
0,session_id,1522
1,Target,석식계
2,Original Data,"(1205, 67)"
3,Missing Values,False
4,Numeric Features,44
5,Categorical Features,21
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(843, 728)"


In [ ]:
compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,46.9060,4.531680e+03,66.5998,0.7729,0.3188,0.1108,1.042
rf,Random Forest Regressor,47.9200,4.552127e+03,67.0247,0.7712,0.5815,0.1113,0.916
gbr,Gradient Boosting Regressor,49.7835,4.817278e+03,68.9091,0.7572,0.8096,0.1133,0.351
huber,Huber Regressor,52.3640,5.206720e+03,71.5951,0.7352,0.8198,0.1194,0.336
br,Bayesian Ridge,53.3464,5.318562e+03,72.3197,0.7301,0.8644,0.1194,0.593
ridge,Ridge Regression,53.6456,5.364311e+03,72.6656,0.7277,0.8763,0.1195,0.036
par,Passive Aggressive Regressor,53.1723,5.390702e+03,72.8303,0.7241,0.8252,0.1210,0.151
omp,Orthogonal Matching Pursuit,53.0467,5.592086e+03,74.2048,0.7158,0.7848,0.1244,0.035
lasso,Lasso Regression,55.7361,5.672898e+03,74.9252,0.7112,0.9059,0.1233,0.040
dt,Decision Tree Regressor,59.7561,7.046071e+03,82.9474,0.6415,0.4063,0.1357,0.038


ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=-1, oob_score=False,
                    random_state=1522, verbose=0, warm_start=False)

In [ ]:
best_5 = compare_models(sort = 'MAE', n_select = 5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,46.9060,4.531680e+03,66.5998,0.7729,0.3188,0.1108,1.044
rf,Random Forest Regressor,47.9200,4.552127e+03,67.0247,0.7712,0.5815,0.1113,0.901
gbr,Gradient Boosting Regressor,49.7835,4.817278e+03,68.9091,0.7572,0.8096,0.1133,0.342
huber,Huber Regressor,52.3640,5.206720e+03,71.5951,0.7352,0.8198,0.1194,0.320
omp,Orthogonal Matching Pursuit,53.0467,5.592086e+03,74.2048,0.7158,0.7848,0.1244,0.036
par,Passive Aggressive Regressor,53.1723,5.390702e+03,72.8303,0.7241,0.8252,0.1210,0.145
br,Bayesian Ridge,53.3464,5.318562e+03,72.3197,0.7301,0.8644,0.1194,0.586
ridge,Ridge Regression,53.6456,5.364311e+03,72.6656,0.7277,0.8763,0.1195,0.036
lasso,Lasso Regression,55.7361,5.672898e+03,74.9252,0.7112,0.9059,0.1233,0.038
dt,Decision Tree Regressor,59.7561,7.046071e+03,82.9474,0.6415,0.4063,0.1357,0.037


In [ ]:
blended = blend_models(estimator_list = best_5, fold = 5, optimize = 'MAE')
pred_holdout = predict_model(blended)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,50.7569,5016.7363,70.8289,0.7853,0.8550,0.1165
1,43.2874,3980.0400,63.0876,0.7755,0.7421,0.0886
2,47.5835,4968.0221,70.4842,0.7392,0.7741,0.1192
3,48.2946,4027.2815,63.4609,0.7430,0.4371,0.1109
4,48.4056,4716.0924,68.6738,0.8006,0.9471,0.1155
Mean,47.6656,4541.6345,67.3071,0.7687,0.7511,0.1101
SD,2.4362,451.2009,3.3753,0.0239,0.1723,0.0111


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,195.101,44914.3906,211.9302,-1.5073,0.9988,0.4221


In [ ]:
final_model = finalize_model(blended)

In [ ]:
pred2 = predict_model(final_model,test)

In [ ]:
submission['석식계'] = pred2['Label']

In [ ]:
submission

,일자,중식계,석식계
0,2021-01-27,1895.175085,-78.874825
1,2021-01-28,1942.075829,-23.381114
2,2021-01-29,1503.380488,-168.402095
3,2021-02-01,1887.259267,-1.405089
4,2021-02-02,1686.313930,-112.200610
5,2021-02-03,1915.756872,-16.179797
6,2021-02-04,1825.078232,-34.618903
7,2021-02-05,1978.741037,66.217854
8,2021-02-08,1909.076847,87.925149
9,2021-02-09,1906.895711,94.309817


In [ ]:
submission.to_csv(path+'baseline_pycaret.csv', index=False)